In [1]:
import pandas as pd
import chess
import chess.pgn
import chess.engine

In [2]:
eng = chess.engine.SimpleEngine.popen_uci("../../stockfish/stockfish")

In [3]:
p = open("../data/RuyLopezMarshall.pgn")

In [4]:
games_id = []
games = []

while True:
    offset = p.tell()
    headers = chess.pgn.read_headers(p)
    if headers is None:
        break
    games_id.append(offset)

for offset in games_id:
    p.seek(offset)
    games.append(chess.pgn.read_game(p))

In [5]:
dados = []
count = -1

for game in games:
    count += 1
    row = {}
    head = game.headers

    if head['Site'].endswith(' INT') or head['Site'] == '?':
        continue

    row['id'] = games_id[count]
    row['Event'] = head['Event']
    row['Site'] = head['Site']
    row['Date'] = head['Date'].split('.')[0]
    row['White'] = head['White']
    row['Black'] = head['Black']
    row['Result'] = head['Result']
    row['BlackElo'] = head['BlackElo']
    row['ECO'] = 'C89'
    row['WhiteElo'] = head['WhiteElo']
    row['White Short Castle'] = 0
    row['White Long Castle'] = 0
    row['Black Short Castle'] = 0
    row['Black Long Castle'] = 0

    #quem ganhou
    if head['Result'] == '0-1':
        row['WhiteWon'] = 0
        row['BlackWon'] = 1
    elif head['Result'] == '1-0':
        row['WhiteWon'] = 1
        row['BlackWon'] = 0
    else:
        row['WhiteWon'] = 0
        row['BlackWon'] = 0

    #qntd de moves
    final_position = game[0].end().board()
    row['Moves'] = final_position.fullmove_number

    #qntd de peças restantes
    final_fen = final_position.board_fen()
    white_count = 0
    black_count = 0
    white_pieces = ['P','N','B','R','Q']
    black_pieces = ['p','n','b','r','q']
    white_has_queen = 0
    black_has_queen = 0
    
    for char in range(0, len(final_fen)):
        if final_fen[char] in white_pieces:
            white_count += 1
        elif final_fen[char] in black_pieces:
            black_count += 1

        if final_fen[char] == 'Q':
            white_has_queen = 1
        elif final_fen[char] == 'q':
            black_has_queen = 1
        
    
    row['WhiteFinalPiecesCount'] = white_count
    row['BlackFinalPiecesCount'] = black_count
    row['FinalPiecesCount'] = white_count + black_count
    
    if white_has_queen == 0 and black_has_queen == 0:
        row['QueenTrade'] = 1
    else:
        row['QueenTrade'] = 0

    
    for move in game[0].mainline_moves():
        eng.analysis

        if move.uci() == 'e1g1':
            row['White Short Castle'] = 1
        elif move.uci() == 'e1c1':
            row['White Long Castle'] = 1
        elif move.uci() == 'e1g1':
            row['Black Short Castle'] = 1
        elif move.uci() == 'e1c1':
            row['Black Long Castle'] = 1
            
        
    dados.append(row)  
     
df = pd.DataFrame(dados)

SyntaxError: invalid syntax (2586299914.py, line 74)

In [ ]:
#corrigir places/Sites
def string_analyzer(row):
    s = row['Site']
    splitted_s = s.split(' ')
    first_part = splitted_s[:-1]
    last_part = splitted_s[-1]
    city = ''

    if last_part.isupper() and len(splitted_s) > 1:
        for w in first_part:
            city += w + ' '
    else:
        city = s

    row['Site'] = city

    return row

df = df.apply(lambda row: string_analyzer(row=row), axis=1)